In [1]:
# API KEY Loading
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_teddynote import logging

logging.langsmith("CH06-Memory")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH06-Memory


# Memory with Chain

In [16]:
from operator import itemgetter
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough, Runnable
from langchain_core.output_parsers import StrOutputParser

In [ ]:
# 모델 정의/초기화 
model = ChatOpenAI()

# 대화형 프롬프트 정의 
prompt = ChatPromptTemplate.from_messages(
    [
        ('system','Your are a helpful assistant.'),
        MessagesPlaceholder(variable_name='chat_history'),
        ('human', '{input}'),
    ]
)

# 메모리 정의/초기화
memory = ConversationBufferMemory(return_messages=True, memory_key='chat_history')

/var/folders/37/2rwxb6k520l4tqxzpx1nh8lh0000gn/T/ipykernel_31628/4009527432.py:14: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(return_messages=True, memory_key='chat_history')


In [8]:
# 초기화된 메모리 확인 
memory.load_memory_variables({})

{'chat_history': []}

In [9]:
# RunnableLambda : 괄호 내 함수 호출 
# memory.load_memory_variables : 메모리에 대화 저장, 결과는 chat_history 변수에 저장 
# itemgetter : chat_history 값을 리스트로 가져오도록 처리 
runnable = RunnablePassthrough.assign(
    chat_history=RunnableLambda(memory.load_memory_variables) 
    | itemgetter('chat_history')
)

In [10]:
# runnable 결과 확인 
runnable.invoke({'input':'Hello'})

{'input': 'Hello', 'chat_history': []}

In [11]:
# 체인 정의
chain = runnable | prompt | model

In [12]:
# 첫번째 대화 
answer = chain.invoke({'input':'안녕하세요. 제 이름은 김영희 입니다. 만나서 반갑습니다.'})
answer.content

'안녕하세요, 김영희님. 만나서 반가워요. 무엇을 도와드릴까요?'

In [13]:
memory.load_memory_variables({})

{'chat_history': []}

In [14]:
# 대화 내용을 메모리에 저장 
memory.save_context(
    {'human':'안녕하세요. 제 이름은 김영희 입니다. 만나서 반갑습니다.'},
    {'ai':answer.content},
)

memory.load_memory_variables({})

{'chat_history': [HumanMessage(content='안녕하세요. 제 이름은 김영희 입니다. 만나서 반갑습니다.', additional_kwargs={}, response_metadata={}),
  AIMessage(content='안녕하세요, 김영희님. 만나서 반가워요. 무엇을 도와드릴까요?', additional_kwargs={}, response_metadata={})]}

In [15]:
# 두번째 대화 
answer = chain.invoke({'input':'제 이름 기억하세요?'})
answer.content

'네, 김영희님이시죠. 어떻게 도와드릴까요?'

# Custom Conversation Chain

In [18]:
class ConversationChainClass(Runnable):

    def __init__(self, llm, prompt, memory, input_key="input"):

        self.prompt = prompt
        self.memory = memory
        self.input_key = input_key

        self.chain = (
            RunnablePassthrough.assign(
                chat_history=RunnableLambda(self.memory.load_memory_variables)
                | itemgetter(memory.memory_key)  # memory_key 와 동일하게 입력
            )
            | prompt
            | llm
            | StrOutputParser()
        )

    def invoke(self, query, configs=None, **kwargs):
        answer = self.chain.invoke({self.input_key: query})
        self.memory.save_context(inputs={"human": query}, outputs={"ai": answer})
        return answer

In [19]:
# 모델 초기화 
model = ChatOpenAI(
    temperature=0,
    model_name='gpt-4o-mini',
)

# 대화형 프롬프트 정의 
prompt = ChatPromptTemplate.from_messages(
    [
        ('system','Your are a helpful assistant.'),
        MessagesPlaceholder(variable_name='chat_history'),
        ('human', '{input}'),
    ]
)

# 메모리 정의/초기화 
memory = ConversationBufferMemory(return_messages=True, memory_key='chat_history')

In [20]:
conversation_chain = ConversationChainClass(model, prompt, memory)

In [21]:
conversation_chain.invoke('안녕하세요. 제 이름은 김영희 입니다. 만나서 반갑습니다.')

'안녕하세요, 김영희님! 만나서 반갑습니다. 어떻게 도와드릴까요?'

In [22]:
conversation_chain.invoke('제 이름 기억하세요?')

'네, 김영희님! 당신의 이름을 기억하고 있습니다. 다른 질문이나 도움이 필요하신 것이 있으면 말씀해 주세요!'

In [23]:
conversation_chain.invoke('오늘 LCEL 문법 수업 듣고 있는데 너무 힘들어요. 힘나는 멘트 주세요!!')

'힘내세요, 김영희님! LCEL 문법 수업은 어려울 수 있지만, 그만큼 여러분의 실력이 쑥쑥 성장하는 과정이에요. 조금씩 나아지는 자신을 믿고, 포기하지 마세요! 모든 노력은 결국 좋은 결과로 돌아올 거예요. 화이팅! 😊'

In [24]:
# 메모리 확인 
conversation_chain.memory.load_memory_variables({})["chat_history"]

[HumanMessage(content='안녕하세요. 제 이름은 김영희 입니다. 만나서 반갑습니다.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='안녕하세요, 김영희님! 만나서 반갑습니다. 어떻게 도와드릴까요?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='제 이름 기억하세요?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='네, 김영희님! 당신의 이름을 기억하고 있습니다. 다른 질문이나 도움이 필요하신 것이 있으면 말씀해 주세요!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='오늘 LCEL 문법 수업 듣고 있는데 너무 힘들어요. 힘나는 멘트 주세요!!', additional_kwargs={}, response_metadata={}),
 AIMessage(content='힘내세요, 김영희님! LCEL 문법 수업은 어려울 수 있지만, 그만큼 여러분의 실력이 쑥쑥 성장하는 과정이에요. 조금씩 나아지는 자신을 믿고, 포기하지 마세요! 모든 노력은 결국 좋은 결과로 돌아올 거예요. 화이팅! 😊', additional_kwargs={}, response_metadata={})]

-----
** End of Documents **